In [18]:
import pandas as pd

In [19]:
pupils_per_class = 21.4 # see Q:\IlMIoDrive\magistrale\2anno\primo_periodo\epi\Data\CSV\2020\Infrastructural\DCIS_SCUOLE_15102020182434230.csv (taken from http://dati.istat.it/#)

In [20]:
code_province = pd.read_csv(r"Q:\IlMIoDrive\magistrale\2anno\primo_periodo\epi\Data\CSV\2020\Administrative\ItalianProvinces.csv")

In [21]:
code_province.head(20)

,code,name,population,abbreviation
0,1,Torino,2252379,TO
1,2,Vercelli,170296,VC
2,3,Novara,368040,NO
3,4,Cuneo,586568,CN
4,5,Asti,213216,AT
5,6,Alessandria,419037,AL
6,7,Aosta,125501,AO
7,8,Imperia,213919,IM
8,9,Savona,274183,SV
9,10,Genova,835829,GE


In [22]:
raw_df = pd.read_csv(r"Q:\IlMIoDrive\magistrale\2anno\primo_periodo\epi\Data\CSV\2020\Infrastructural\EDICONSISTENZASTA20181920180901.csv")

In [25]:
raw_df.head()

,ANNOSCOLASTICO,CODICESCUOLA,CODICEEDIFICIO,SPERFICIEAREATOTALE,SPERFICIEAREALIBERA,VOLUME
0,201819,FGEE82302X,710140317,"2.310,00","4.915,00",7000
1,201819,COTD00101T,131450925,"4.800,00",-,35000
2,201819,CTAA08305Q,870550868,"130,00",-,435
3,201819,MITF11000E,151462103,"19.224,00",-,9068
4,201819,PGPC01801G,540352633,-,-,-


In [29]:
raw_df = raw_df[raw_df["VOLUME"] != "-"]

In [30]:
raw_df["VOLUME"] = [float(volume) for volume in raw_df["VOLUME"].tolist()]

In [31]:
raw_df.head()

,ANNOSCOLASTICO,CODICESCUOLA,CODICEEDIFICIO,SPERFICIEAREATOTALE,SPERFICIEAREALIBERA,VOLUME
0,201819,FGEE82302X,710140317,"2.310,00","4.915,00",7000.0
1,201819,COTD00101T,131450925,"4.800,00",-,35000.0
2,201819,CTAA08305Q,870550868,"130,00",-,435.0
3,201819,MITF11000E,151462103,"19.224,00",-,9068.0
5,201819,CSIS06100T,780031081,"17.000,00",-,18600.0


In [14]:
raw_df.head(20)

,ANNOSCOLASTICO,CODICESCUOLA,CODICEEDIFICIO,SPERFICIEAREATOTALE,SPERFICIEAREALIBERA,VOLUME
0,201819,FGEE82302X,710140317,"2.310,00","4.915,00",7000
1,201819,COTD00101T,131450925,"4.800,00",-,35000
2,201819,CTAA08305Q,870550868,"130,00",-,435
3,201819,MITF11000E,151462103,"19.224,00",-,9068
4,201819,PGPC01801G,540352633,-,-,-
5,201819,CSIS06100T,780031081,"17.000,00",-,18600
6,201819,ALEE818052,61410001,"1.400,00","1.000,00",3360
7,201819,MORC004013,360300326,"4.944,50","1.650,00",40522.35
8,201819,CLAA82002B,850180536,"1.537,00","1.120,00",1827.49
9,201819,PCAA819033,330170066,"500,00",-,1650
